In [1]:
import os, sys
%pylab inline
import face_recognition
from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image, ImageDraw
import pandas as pd
from IPython.display import display
import pathlib

Populating the interactive namespace from numpy and matplotlib


### Set up Paths

In [2]:
DATASET_PATH = os.path.abspath('')
CELEB_FACES_DATASET_FILE = pathlib.Path((pathlib.Path(DATASET_PATH).parent), "Celeb_Faces","list_attr_celeba.csv")
CELEB_IMAGE_PATH = pathlib.Path((pathlib.Path(DATASET_PATH).parent), "Celeb_Faces","img_align_celeba","img_align_celeba")

### Load csv data


In [3]:
celeb_faces_dataframe = pd.read_csv(CELEB_FACES_DATASET_FILE)
celeb_faces_dataframe.head(1)

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,-1,1,1,-1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1


In [4]:
# Select only the image id and the attractiveness
image_path_attractive_dataframe = celeb_faces_dataframe[['image_id','Attractive']]
# convert image_id from 00001.jpg to PATH/00001.jpg
image_path_attractive_dataframe['image_id'] = image_path_attractive_dataframe['image_id'].map(lambda filename: pathlib.Path(CELEB_IMAGE_PATH,filename))

c:\program files (x86)\microsoft visual studio\shared\python36_64\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [5]:
# rename column image_id -> image_path & Attractive -> attractive
image_path_attractive_dataframe.columns = ["image_path", 'attractive']

In [6]:
image_path_attractive_dataframe.head(1)

,image_path,attractive
0,C:\Users\James H\git\TinderBot\Database\Datase...,1


## Create new Dataframe with Normalized positions of landmarks for each face

In [7]:
landmarks_list = []
for idx, image_path in enumerate(image_path_attractive_dataframe['image_path']):
    image = face_recognition.load_image_file(image_path)
    landmarks = face_recognition.face_landmarks(image)
    attractive = image_path_attractive_dataframe['attractive'][idx]
    # check if no face detected
    if (landmarks == []):
        pass
    else:
        # normalise landmarks
        landmarks = landmarks[0]
#         print(image.shape[0:2])
        max_y, max_x = image.shape[0:2]
#         print(landmarks)
        for landmark, points in landmarks.items():
            normalized_points = []
            for point in points:
                norm_x = (point[0])/max_x
                norm_y = (point[1])/max_y
                normalized_point = (norm_x, norm_y)
                normalized_points.append(normalized_point)
            landmarks[landmark] = normalized_points
        # add attractiveness as a key 
        landmarks['attractive'] = attractive
        landmarks_list.append(landmarks)
        
# create a datframe where columns are landmark features and attractive 
landmarks_attractive_df = pd.DataFrame(landmarks_list)
landmarks_attractive_df

,chin,left_eyebrow,right_eyebrow,nose_bridge,nose_tip,left_eye,right_eye,top_lip,bottom_lip,attractive
0,"[(0.33146067415730335, 0.518348623853211), (0....","[(0.3202247191011236, 0.44954128440366975), (0...","[(0.5, 0.48623853211009177), (0.55617977528089...","[(0.4550561797752809, 0.5229357798165137), (0....","[(0.4044943820224719, 0.6559633027522935), (0....","[(0.3539325842696629, 0.5045871559633027), (0....","[(0.5449438202247191, 0.536697247706422), (0.5...","[(0.398876404494382, 0.6926605504587156), (0.4...","[(0.6123595505617978, 0.7110091743119266), (0....",1
1,"[(0.3089887640449438, 0.5091743119266054), (0....","[(0.3089887640449438, 0.47706422018348627), (0...","[(0.5112359550561798, 0.481651376146789), (0.5...","[(0.47191011235955055, 0.5137614678899083), (0...","[(0.4101123595505618, 0.6376146788990825), (0....","[(0.34831460674157305, 0.5091743119266054), (0...","[(0.550561797752809, 0.5275229357798165), (0.5...","[(0.37640449438202245, 0.6926605504587156), (0...","[(0.6123595505617978, 0.7018348623853211), (0....",-1
2,"[(0.2303370786516854, 0.5779816513761468), (0....","[(0.25842696629213485, 0.481651376146789), (0....","[(0.5449438202247191, 0.43577981651376146), (0...","[(0.4887640449438202, 0.48623853211009177), (0...","[(0.4438202247191011, 0.6009174311926605), (0....","[(0.3202247191011236, 0.5321100917431193), (0....","[(0.5730337078651685, 0.5137614678899083), (0....","[(0.398876404494382, 0.7064220183486238), (0.4...","[(0.5842696629213483, 0.7018348623853211), (0....",1
3,"[(0.21910112359550563, 0.5), (0.22471910112359...","[(0.29775280898876405, 0.45871559633027525), (...","[(0.5617977528089888, 0.463302752293578), (0.5...","[(0.5112359550561798, 0.5091743119266054), (0....","[(0.47752808988764045, 0.6284403669724771), (0...","[(0.34831460674157305, 0.5137614678899083), (0...","[(0.550561797752809, 0.5229357798165137), (0.5...","[(0.42134831460674155, 0.7110091743119266), (0...","[(0.5730337078651685, 0.7064220183486238), (0....",1
4,"[(0.24719101123595505, 0.5), (0.25842696629213...","[(0.2808988764044944, 0.48623853211009177), (0...","[(0.5224719101123596, 0.4724770642201835), (0....","[(0.48314606741573035, 0.5137614678899083), (0...","[(0.4438202247191011, 0.6422018348623854), (0....","[(0.34269662921348315, 0.5229357798165137), (0...","[(0.5561797752808989, 0.518348623853211), (0.5...","[(0.4101123595505618, 0.7018348623853211), (0....","[(0.5842696629213483, 0.7018348623853211), (0....",1
...,...,...,...,...,...,...,...,...,...,...
196994,"[(0.2752808988764045, 0.5), (0.275280898876404...","[(0.30337078651685395, 0.47706422018348627), (...","[(0.550561797752809, 0.4954128440366973), (0.5...","[(0.4943820224719101, 0.5275229357798165), (0....","[(0.4550561797752809, 0.6513761467889908), (0....","[(0.34269662921348315, 0.5137614678899083), (0...","[(0.5674157303370787, 0.5275229357798165), (0....","[(0.4101123595505618, 0.6972477064220184), (0....","[(0.5842696629213483, 0.6972477064220184), (0....",1
196995,"[(0.2303370786516854, 0.48623853211009177), (0...","[(0.2696629213483146, 0.48623853211009177), (0...","[(0.5393258426966292, 0.48623853211009177), (0...","[(0.4887640449438202, 0.5229357798165137), (0....","[(0.43820224719101125, 0.6559633027522935), (0...","[(0.33146067415730335, 0.518348623853211), (0....","[(0.5674157303370787, 0.5275229357798165), (0....","[(0.3707865168539326, 0.6880733944954128), (0....","[(0.6235955056179775, 0.6880733944954128), (0....",-1
196996,"[(0.2303370786516854, 0.5229357798165137), (0....","[(0.3146067415730337, 0.4954128440366973), (0....","[(0.5617977528089888, 0.5137614678899083), (0....","[(0.5112359550561798, 0.5275229357798165), (0....","[(0.4550561797752809, 0.6467889908256881), (0....","[(0.3595505617977528, 0.5321100917431193), (0....","[(0.5730337078651685, 0.5458715596330275), (0....","[(0.38202247191011235, 0.6926605504587156), (0...","[(0.6123595505617978, 0.7018348623853211), (0....",-1
196997,"[(0.25280898876404495, 0.5275229357798165), (0...",

### Save Dataframe

In [10]:
landmarks_attractive_df.to_pickle("./landmarks_attractive_dataframe.pkl")